# Setup

In [1]:
%%sh
pip install -q dash
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table

In [2]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.0M  100 14.0M    0     0  24.5M      0 --:--:-- --:--:-- --:--:-- 24.5M


# Dash

In [3]:
%%writefile dash_app.py

import dash
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

df = pd.read_csv('df_vacc.csv', index_col=0, parse_dates=True)
df.index = pd.to_datetime(df['date'])


import plotly.graph_objects as go # or plotly.express as px
fig = px.line(df, x=df.index, 
              y= df["weekly"],
              color='codelocation',
              template='plotly_dark').update_layout(
                  {'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                   'paper_bgcolor': 'rgba(0, 0, 0, 0)'})

config = dict({'scrollZoom': True})
              

# Creates a list of dictionaries, which have the keys 'label' and 'value'.
def get_options(list_location):
  dict_list = []
  for i in list_location:
    dict_list.append({'label': i, 'value': i})
  return dict_list


app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.Div(className='row',
                 children=[
                    html.Div(className='four columns div-user-controls',
                             children=[
                                 html.H2('COVID DASHBOARD'),
                                 html.P('Visualising time series with Plotly - Dash.'),
                                 html.P('Pick one or more location from the dropdown below.'),
                                 html.Div(className='div-for-dropdown',
                                          children=[
                                                    dcc.Dropdown(id='locselector',
                                                                 options=get_options(df['codelocation'].unique()),
                                                                 multi=True,
                                                                 value=[df['codelocation'].sort_values()[0]],
                                                                 style={'backgroundColor': '#1E1E1E'},
                                                                 className='locselector')
                                                    ],
                                          style={'color': '#1E1E1E'})
                                 ]
                             ),
                    html.Div(className='eight columns div-for-charts bg-grey',
                             children=[
    dcc.Graph(id='timeseries',
              config={'displayModeBar': False},
              animate=True,
              figure=fig)
])
                    
                    ])
                              ])

app.run_server(debug=True)

Writing dash_app.py


In [4]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [5]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://f5e07325de20.ngrok.io


In [6]:
!python dash_app.py

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
